## Category X Analysis Workbook Template
**Written by: INSERT NAME**

In [1]:
%pip install -r requirements.txt --upgrade

  Using cached gspread-6.2.0-py3-none-any.whl.metadata (11 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
from p2utilityfunctions import solvercomplete, nutrition
#for documentation on any function run "function?", ex nutrition?
import numpy as np
import pandas as pd
from eep153_tools.sheets import read_sheets

### Part 1: Setting constraints and general variables

In [3]:
age = 25 # change this depending on prison average
sex = "F" #change this as well; M for Male and F for Female
activity_level = "Moderately Active" #change this; can be Moderately Active, Sedentary, or Active

In [4]:
# time to make minimizing and maximizing constraints
bmin, bmax = nutrition(age, sex, activity_level)
print(f"Current minimizing constraints are: \n \n {bmin}")
print(f"Current maximum constraints are: \n \n {bmax}")

Current minimizing constraints are: 
 
 Nutrient
Energy            2200.0
Protein             46.0
Carbohydrate       130.0
Dietary Fiber       28.0
Linoleic Acid       12.0
Linolenic Acid       1.1
Calcium           1000.0
Iron                18.0
Magnesium          310.0
Phosphorus         700.0
Potassium         4700.0
Zinc                 8.0
Copper               0.9
Selenium            55.0
Vitamin A          700.0
Vitamin E           15.0
Vitamin D           15.0
Vitamin C           75.0
Thiamin              1.1
Riboflavin           1.1
Niacin              14.0
Vitamin B6           1.3
Vitamin B12          2.4
Choline            425.0
Vitamin K           90.0
Folate             400.0
Name: Female_19_30, dtype: float64
Current maximum constraints are: 
 
 Nutrient
Sodium    2300.0
Name: Female_19_30, dtype: float64


/home/jovyan/EEP-153-Project-2/p2utilityfunctions.py:41: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  bmin[0] = active_energy


#### A Note On Case Specific Constraint Editing
Feel free to add in other maximizing constraints if you think they are relevant. Just make sure to justify them

### Part 2: Determining recipe and nutrition set

**Nutrition set is what constrains the options, since we are using a left join (or merge) on nutrients and recipes. The Nutrition dataset provides nutritional values and the recipe set provides weights**

In [5]:
data_url = "https://docs.google.com/spreadsheets/d/1xqixhrAoDq9rWJf_FC3Y2eXdd010DTLPCS7JJMCfwP8/edit?usp=sharing"
recipes = read_sheets(data_url, sheet="recipes")
nutrients = read_sheets(data_url, sheet="nutrients")

In [6]:
nutrients.head()

,ingred_code,Ingredient description,Capric acid,Lauric acid,Myristic acid,Palmitic acid,Palmitoleic acid,Stearic acid,Oleic acid,Linoleic Acid,...,Vitamin B12,"Vitamin B-12, added",Vitamin B6,Vitamin C,Vitamin D,Vitamin E,"Vitamin E, added",Vitamin K,Water,Zinc
0,1001,"Butter, salted",2.529,2.587,7.436,21.697,0.961,9.999,19.961,2.728,...,0.17,0.0,0.003,0.0,0.0,2.32,0.0,7.0,15.87,0.09
1,1002,"Butter, whipped, with salt",2.039,2.354,7.515,20.531,1.417,7.649,17.370,2.713,...,0.07,0.0,0.008,0.0,0.0,1.37,0.0,4.6,16.72,0.05
2,1003,"Butter oil, anhydrous",2.495,2.793,10.005,26.166,2.228,12.056,25.026,2.247,...,0.01,0.0,0.001,0.0,0.0,2.80,0.0,8.6,0.24,0.01
3,1004,"Cheese, blue",0.601,0.491,3.301,9.153,0.816,3.235,6.622,0.536,...,1.22,0.0,0.166,0.0,0.5,0.25,0.0,2.4,42.41,2.66
4,1005,"Cheese, brick",0.585,0.482,3.227,8.655,0.817,3.455,7.401,0.491,...,1.26,0.0,0.065,0.0,0.5,0.26,0.0,2.5,41.11,2.60


**Now, drop any foods within ingredients that you want in order to constrain to the specific area of interest (ex. particular population)**

Below is a search function. Feel free to use it to either view particular foods within the dataset or set cut = True if you want to simply cut out all food options in the final result that have the search term in it.

In [7]:
def nutrient_search(search_term, nutrients, cut = False):
    """
    Filters the nutrients DataFrame based on the presence or absence of a search term in the 'Ingredient description' column.

    When 'cut' is True, the function returns only the rows where the 'Ingredient description' contains the given search term.
    When 'cut' is False, it returns only the rows where the 'Ingredient description' does NOT contain the search term.

    Parameters:
        search_term (str): The term to search for in the 'Ingredient description' column.
        nutrients (pd.DataFrame): The DataFrame containing nutrient data with an 'Ingredient description' column.
        cut (bool, optional): Determines the filtering mode. If True, select rows containing the search term.
                              If False (default), select rows that do not contain the search term.

    Returns:
        pd.DataFrame: The filtered DataFrame based on the specified condition.
    """
    if cut:
        return nutrients[~nutrients['Ingredient description'].str.contains(search_term)]
    else:
        return nutrients[nutrients['Ingredient description'].str.contains(search_term)]

In [8]:
#example use
nutrient_search('Butter', nutrients).head()

,ingred_code,Ingredient description,Capric acid,Lauric acid,Myristic acid,Palmitic acid,Palmitoleic acid,Stearic acid,Oleic acid,Linoleic Acid,...,Vitamin B12,"Vitamin B-12, added",Vitamin B6,Vitamin C,Vitamin D,Vitamin E,"Vitamin E, added",Vitamin K,Water,Zinc
0,1001,"Butter, salted",2.529,2.587,7.436,21.697,0.961,9.999,19.961,2.728,...,0.17,0.0,0.003,0.0,0.0,2.32,0.0,7.0,15.87,0.09
1,1002,"Butter, whipped, with salt",2.039,2.354,7.515,20.531,1.417,7.649,17.370,2.713,...,0.07,0.0,0.008,0.0,0.0,1.37,0.0,4.6,16.72,0.05
2,1003,"Butter oil, anhydrous",2.495,2.793,10.005,26.166,2.228,12.056,25.026,2.247,...,0.01,0.0,0.001,0.0,0.0,2.80,0.0,8.6,0.24,0.01
87,1145,"Butter, without salt",2.529,2.587,7.436,21.697,1.820,9.999,20.400,1.830,...,0.17,0.0,0.003,0.0,0.0,2.32,0.0,7.0,17.94,0.09
411,4601,"Butter, light, stick, with salt",1.380,1.550,5.547,14.499,1.237,6.682,13.867,1.244,...,0.13,0.0,0.010,0.0,0.0,1.58,0.0,4.8,42.10,0.26


## Running the optimization
Here is a call to the optimization function

In [11]:
solvercomplete(sex, age,recipes, nutrients, bmin, bmax)

Cost of diet for F's of age 25 is $2.46 per day.

The diet will consist of (in 100s of grams or milliliters):
Milk, low fat (1%)                          6.93
Carp, steamed or poached                    0.08
Egg, yolk only, raw                         0.10
Split peas, from dried, fat added           5.04
Rice, white, cooked, made with margarine    1.66
Cereal, rice flakes                         0.07
Cereal, toasted oat                         0.18
Ripe plantain, raw                          3.52
dtype: float64

With the following nutritional outcomes of interest:
                    Outcome  Recommendation
Nutrient                                   
Energy               2200.0          2200.0
Protein           75.219377            46.0
Carbohydrate     294.444726           130.0
Dietary Fiber     49.347705            28.0
Linoleic Acid     27.669828            12.0
Linolenic Acid     3.394831             1.1
Calcium         1046.009687          1000.0
Iron                   18.0      

'2.46'